<a href="https://colab.research.google.com/github/ttvhh/CS114.K21/blob/master/CS114_Lab_04_%08CloudVisionAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CS114 - BÀI TẬP THỰC HÀNH LAB-04
**@Lê Đình Duy**

Đây là tài liệu hướng dẫn thực hành về phát triển ứng dụng sử dụng Google Cloud Vision API




#Làm quen với Google Cloud Platform (GCP) - Các bước chính

*(SV tự tìm hiểu)*

* Tạo project từ Google Cloud Console http://console.cloud.google.com/ - đặt tên là CS114-CloudVisionAPI-Demo-MSSV
* Kích hoạt Vision API (và AutoML API) https://console.cloud.google.com/apis/dashboard
* Xem Dashboard của Vision API https://console.cloud.google.com/apis/api/vision.googleapis.com/
* Tạo API Key https://console.cloud.google.com/apis/credentials 
* Tạo Service Account https://cloud.google.com/docs/authentication#service_accounts
* Tham khảo tài liệu của Vision API https://cloud.google.com/vision/docs 

Tham khảo
* https://cloud.google.com/vision/docs/setup

**Hướng dẫn:**

*Để sử dụng các dịch vụ của GCP, cần phải tạo project.*

A project organizes all your GCP resources. A project consists of the following components:
* a set of collaborators
* enabled APIs (and other resources)
* monitoring tools
* **billing information - cần phải có credit card**
* authentication and access controls

**You have THREE OPTIONS for calling the Vision API:**

* Google supported client libraries (recommended) - Python https://googleapis.dev/python/vision/latest/index.html
* REST https://cloud.google.com/vision/docs/reference/rest
* gRPC https://cloud.google.com/vision/docs/reference/rpc

The client libraries are available for several popular languages. For information on installing the client libraries, see Vision API Client Libraries.


# Sử dụng Command Line (curl) với API_KEY

Trong phần này, chúng ta sẽ thử tính năng Image Annotation của Goolge Vision API. Để có thể sử dụng được API, cần phải có API_KEY và hiểu cách gọi tương ứng qua REST API

Tham khảo
* https://cloud.google.com/vision/docs/quickstart-cli
* https://becominghuman.ai/how-to-use-google-cloud-vision-api-4fbd800641f9

**Ảnh input**

![Shanghai](https://cloud.google.com/vision/docs/images/shanghai_small.jpeg)

**Create the request JSON**

*Create the JSON request file with the following text, and save it as a `request.json` plain text file in your working directory:*

```
{
  "requests": [
    {
      "image": {
        "source": {
          "imageUri": "gs://cloud-samples-data/vision/using_curl/shanghai.jpeg"
        }
      },
      "features": [
        {
          "type": "LABEL_DETECTION",
          "maxResults": 3
        },
        {
          "type": "OBJECT_LOCALIZATION",
          "maxResults": 1
        },
        {
          "type": "TEXT_DETECTION",
          "maxResults": 1,
          "model": "builtin/latest"
        }
      ]
    }
  ]
}
```



**Send the request**

*You use curl and the body content from request.json to send the request to the Vision API*

In [0]:
import getpass

YOUR_API_KEY = getpass.getpass('Enter your API_KEY')

!curl -v -s -H "Content-Type: application/json" \
    https://vision.googleapis.com/v1/images:annotate?key=$YOUR_API_KEY \
    --data-binary @request.json

**Interpret the response**

*You should see a JSON response similar to the one below*

```
{
  "responses": [
    {
      "labelAnnotations": [
        {
          "mid": "/m/09g5pq",
          "description": "People",
          "score": 0.9504782,
          "topicality": 0.9504782
        },
        {
          "mid": "/m/01c8br",
          "description": "Street",
          "score": 0.8911568,
          "topicality": 0.8911568
        },
        {
          "mid": "/m/079bkr",
          "description": "Mode of transport",
          "score": 0.89089024,
          "topicality": 0.89089024
        }
      ],
      "textAnnotations": [
        {
          "locale": "zh",
          "description": "牛牛面馆\n",
          "boundingPoly": {
            "vertices": [
              {
                "x": 159,
                "y": 212
              },
              {
                "x": 947,
                "y": 212
              },
              {
                "x": 947,
                "y": 354
              },
              {
                "x": 159,
                "y": 354
              }
            ]
          }
        },
        ...
      ],
      "fullTextAnnotation": {
        "pages": [
          {
            ...
                "paragraphs": [
                  {
                    ...
                    "words": [
                      {
                        ...
                        "symbols": [
                          {
                            ...
                ],
                "blockType": "TEXT"
              }
            ]
          }
        ],
        "text": "牛牛面馆\n"
      },
      "localizedObjectAnnotations": [
        {
          "mid": "/m/01g317",
          "name": "Person",
          "score": 0.94413143,
          "boundingPoly": {
            "normalizedVertices": [
              {
                "x": 0.26063988,
                "y": 0.46869153
              },
              {
                "x": 0.40736017,
                "y": 0.46869153
              },
              {
                "x": 0.40736017,
                "y": 0.8957791
              },
              {
                "x": 0.26063988,
                "y": 0.8957791
              }
            ]
          }
        }
      ]
    }
  ]
}
```



**Kết quả - Label Detection**

![Label detection result](https://cloud.google.com/vision/docs/images/shanghai_labels.png)


**Kết quả - Object Detection**

![Object detection result](https://cloud.google.com/vision/docs/images/shanghai_object.png)

#Ứng dụng phát hiện khuôn mặt (face detection)

*For your convenience, the Vision API can perform feature detection directly on an image file located in Google Cloud Storage or on the Web without the need to send the contents of the image file in the body of your request.*

Tham khảo:
* http://temp.theadora.io/google-cloud-python/vision/gapic/api.html
* https://www.geeksforgeeks.org/how-to-use-vision-api-from-google-cloud/
* https://google-cloud-python.readthedocs.io/en/0.32.0/vision/gapic/v1/types.html

## Authentication với Service Account

In [0]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(os.curdir, 'creds.json') 

!echo $GOOGLE_APPLICATION_CREDENTIALS

!cat $GOOGLE_APPLICATION_CREDENTIALS

## Install Client Library

In [0]:
!pip install --upgrade google-cloud-vision

## Hiển thị ảnh

*Viết lại thành hàm và nhập liệu từ form*


In [0]:
from google.colab.patches import cv2_imshow
import cv2
import os

def download_view(img_url):
  img_file = os.path.basename(img_url)
  !curl -o $img_file $img_url
  img = cv2.imread(img_file, cv2.IMREAD_UNCHANGED)
  cv2_imshow(img)
  return img_file

In [0]:
#@title Nhập URL

img_url = "https://i-vnexpress.vnecdn.net/2020/03/25/my-1-jpeg-2570-1585098851.jpg" #@param {type:"string"}

print(img_url)
img_file = download_view(img_url)
print(img_file)

## Cài đặt hàm detect_face

In [0]:
# https://cloud.google.com/vision/docs/face-tutorial

def detect_face(face_file, max_results=20):
    """Uses the Vision API to detect faces in the given file.

    Args:
        face_file: A file-like object containing an image with faces.

    Returns:
        An array of Face objects with information about the picture.
    """
    client = vision.ImageAnnotatorClient()

    content = face_file.read()
    image = types.Image(content=content)

    return client.face_detection(
        image=image, max_results=max_results).face_annotations

## Cài đặt hàm highlight_faces

In [0]:
# https://cloud.google.com/vision/docs/face-tutorial

def highlight_faces(image, faces, output_filename):
    """Draws a polygon around the faces, then saves to output_filename.

    Args:
      image: a file containing the image with the faces.
      faces: a list of faces found in the file. This should be in the format
          returned by the Vision API.
      output_filename: the name of the image file to be created, where the
          faces have polygons drawn around them.
    """
    im = Image.open(image)
    draw = ImageDraw.Draw(im)
    # Sepecify the font-family and the font-size
    for face in faces:
        box = [(vertex.x, vertex.y)
               for vertex in face.bounding_poly.vertices]
        draw.line(box + [box[0]], width=5, fill='#00ff00')
        # Place the confidence value/score of the detected faces above the
        # detection box in the output image
        draw.text(((face.bounding_poly.vertices)[0].x,
                   (face.bounding_poly.vertices)[0].y - 30),
                  str(format(face.detection_confidence, '.3f')) + '%',
                  fill='#FF0000')
    im.save(output_filename)

## Tổng hợp lại

Lưu ý: 
* Paste link đến tập tin ảnh cần detect faces ở form trên
* Kết quả sẽ hiển thị trong ảnh xxx.facedet.jpg

In [0]:
from google.cloud import vision
from google.cloud.vision import types
from PIL import Image, ImageDraw
from google.colab.patches import cv2_imshow
import cv2

output_filename = img_file + '.facedet.jpg'
max_results = 10

with open(img_file, 'rb') as image:
    faces = detect_face(image, max_results)
    print('Found {} face{}'.format(
        len(faces), '' if len(faces) == 1 else 's'))

    print('Writing to file {}'.format(output_filename))
    # Reset the file pointer, so we can read the file again
    image.seek(0)
    highlight_faces(image, faces, output_filename)
    img = cv2.imread(output_filename, cv2.IMREAD_UNCHANGED)
    cv2_imshow(img)


## Minh hoạ Face detection từ URI

In [0]:
# https://cloud.google.com/vision/docs/detecting-faces

from google.cloud import vision

def detect_faces_uri(uri):
    """Detects faces in the file located in Google Cloud Storage or the web."""
    client = vision.ImageAnnotatorClient()
    image = vision.types.Image()
    image.source.image_uri = uri

    response = client.face_detection(image=image)
    faces = response.face_annotations

    # Names of likelihood from google.cloud.vision.enums
    likelihood_name = ('UNKNOWN', 'VERY_UNLIKELY', 'UNLIKELY', 'POSSIBLE',
                       'LIKELY', 'VERY_LIKELY')
    print('Faces:')

    for face in faces:
        print('anger: {}'.format(likelihood_name[face.anger_likelihood]))
        print('joy: {}'.format(likelihood_name[face.joy_likelihood]))
        print('surprise: {}'.format(likelihood_name[face.surprise_likelihood]))

        vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in face.bounding_poly.vertices])

        print('face bounds: {}'.format(','.join(vertices)))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
     

In [0]:
detect_faces_uri(img_url)